# Exercício 2 - Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $N>0$
```python
INPUT  a, b : Int
assume  a > 0 and b > 0 and a < N and b < N
r, r', s, s', t, t' = a, b, 1, 0, 0, 1
while r' != 0
  q = r div r'
  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
OUTPUT r, s, t
```

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $\gcd$  é   $\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$ .
2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”. 
4. Prove a correção  do programa LPA em ambas as codificações.

# Exercício 2 - Solução

```python
assert r = gcd(a,b) and r_prime = 0 and exists s,t. gcd(a,b) = a * s + b * t
```

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import *

def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

```python
INPUT  a, b : Int
assume  a > 0 and b > 0 and a < N and b < N
r, r', s, s', t, t', rP, sP, tP = a, b, 1, 0, 0, 1, a,1,0
while r' != 0
  q = r div r'
  r, r', s, s', t, t' = r', rP − q × r', s', sP − q × s', t', tP − q × t'
  rP = r
  sP = s
  tP = t
OUTPUT r, s, t
```

``` python

Programa de fluxos.

Sejam pre <- a > 0 and b > 0 and a < N and b < N
      inv <- (r > 0) and (r < N) and (exists s,t. r = a*s + b*t)
      pos <- r = gcd(a,b) and r_prime = 0 and exists s,t. gcd(a,b) = a * s + b * t

        
assume pre;
r := a, r' := b, s := 1, s' := 0, t := 0, t' := 1;
assert inv;
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';
((assume r' != 0 and inv; q = r div r'; r := r', r' := r - q * r', s := s', s' := s - q * s', t := t', t' := t - q * t';
assert inv; assume False) || assume(r' = 0) and inv)
assert pos;
```

Denotação lógica com WPC.
```python
[
assume pre;
r := a; r' := b; s := 1; s' := 0; t := 0; t' := 1;
assert inv;
havoc {r, r', s, s', t, t'};
((assume r' != 0 and inv; q = r div r'; r := r'; r' := r - q * r'; s := s'; s' := s - q * s'; t := t'; t' := t - q * t';
 assert inv; assume False) || assume(r' = 0) and inv)
assert pos;
]
=
pre -> [ assert inv; havoc q,r, r',s,s',t,t'; ...; assert pos ] [a/r][b/r'][1/s][0/s'][0/t][1/t']
=
pre -> inv[a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t] and (forall r,r',s,s',t,t',rP,sP,tP. [...; assert pos]) [a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t]
=
pre -> inv[a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t] and
       (forall r,r',s,s',t,t',rP,sP,tP.
           ( (r' != 0 and inv) -> inv[r/rP][s/sP][t/tP][(tP-q*t')/t'][t'/t][(sP-q*s')/s'][s'/s][(rP-q*r')/r'][r'/r][(r/r')/q]) )
             and
             ( (not(r' != 0) and inv) -> pos)
       )
```


In [ ]:
gcd = Symbol('gcd', FunctionType(INT,[INT,INT]))
a = Symbol('a',INT)
b = Symbol('b',INT)
ax1 = Equals(gcd(a, Int(0)), a)
ax2 = ForAll([a, b], Ite(GE(a, b),
                        Equals(gcd(a, b), gcd(Minus(a, b), b)),
                        Equals(gcd(a, b), gcd(a, Minus(b, a)))
                    )
            )
ax2_a = ForAll([a, b], Ite(GE(a, b),
                                 Equals(gcd(a, b), gcd(b, Minus(a, Times(b, Div(a,b))))),
                                 Equals(gcd(a, b), gcd(b,a))
                        )
            )
axioms = And(ax1, ax2)

In [ ]:

prove(Implies(axioms,Equals(gcd(Int(5),Int(3)),Int(1))))

In [ ]:

r = Symbol('r', INT)
r_prime = Symbol('r_prime', INT)
s = Symbol('s', INT)
s_prime = Symbol('s_prime', INT)
t = Symbol('t', INT)
t_prime = Symbol('t_prime', INT)
rP = Symbol('rP', INT)
sP = Symbol('sP', INT)
tP = Symbol('tP', INT)
N = Symbol('N', INT)
q = Symbol('q', INT)
aux = Symbol('aux', INT)


pre = And(GT(a, Int(0)), GT(b, Int(0)), GT(N, a), GT(N, b))
pos = And(ForAll([aux], Implies(GT(r, aux), Not(Equals(aux, Plus(Times(a, s), Times(b, t)))))), Equals(r, Plus(Times(a, s), Times(b, t))))
inv = And(GT(r, Int(0)), GT(N, r), Equals(r, Plus(Times(a, s), Times(b, t))))


ini = substitute(inv, {r:a, r_prime:b, s:Int(1), s_prime:Int(0), t:Int(0), t_prime:Int(1), rP:r, sP:s, tP:t})
pres = Implies(And(Not(Equals(r_prime, Int(0))), inv), substitute(substitute(inv, {t_prime: Minus(tP, Times(q, t_prime))}, {t: t_prime}, {s_prime: Minus(sP, Times(q, s_prime))}, {s: s_prime}, {r_prime: Minus(rP, Times(q, r_prime))}, {r: r_prime}), {q: Div(r, r_prime)}))
util = Implies(And(Equals(r_prime, Int(0)), inv), pos)
vc = Implies(pre, And(ini, ForAll([r,r_prime,s,s_prime,t,t_prime,rP,sP,tP], And(pres, util))))
print("<h3>WPC</h3>")
prove(Implies(axioms, vc))

Denotação lógica com SPC
```python
[
assume pre;
r := a; r' := b; s := 1; s' := 0; t := 0; t' := 1; 
assert inv;
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';
((assume r' != 0 and inv; q = r / r'; r := r'; r' := r - q * r'; s := s'; s' := s - q * s'; t := t'; t' := t - q * t';
rP := r; sP := s; tP := t assert inv; assume False) || assume(r' = 0) and inv)
assert pos;
]
=> 
[
assume pre;
r := a; r' := b; s := 1; s' := 0; t := 0; t' := 1; 
assert inv;
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';
((assume r' != 0 and inv; q = r / r'; r := r'; r' := r - q * r'; s := s'; s' := s - q * s'; t := t'; t' := t - q * t';
rP := r; sP := s; tP := t assert inv; assume False) || assume(r' = 0) and inv)
]
-> pos
=>
[
assume pre;
r := a; r' := b; s := 1; s' := 0; t := 0; t' := 1;
assert inv;
havoc r, r', s, s', t, t', rP, sP, tP;



In [ ]:
r = Symbol('r', INT)
r_prime = Symbol('r_prime', INT)
s = Symbol('s', INT)
s_prime = Symbol('s_prime', INT)
t = Symbol('t', INT)
t_prime = Symbol('t_prime', INT)
rP = Symbol('rP', INT)
sP = Symbol('sP', INT)
tP = Symbol('tP', INT)
N = Symbol('N', INT)
q = Symbol('q', INT)
aux = Symbol('aux', INT)


pre = And(GT(a, Int(0)), GT(b, Int(0)), GT(N, a), GT(N, b))
pos = And(ForAll([aux], Implies(GT(r, aux), Not(Equals(aux, Plus(Times(a, s), Times(b, t)))))), Equals(r, Plus(Times(a, s), Times(b, t))))
inv = And(GT(r, Int(0)), GT(N, r), Equals(r, Plus(Times(a, s), Times(b, t))))


ini = substitute(inv, {r:a, r_prime:b, s:Int(1), s_prime:Int(0), t:Int(0), t_prime:Int(1), rP:r, sP:s, tP:t})
pres = Implies(And(Not(Equals(r_prime, Int(0))), inv), substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(inv, {tP:t}), {sP:s}), {rP:r}), {t_prime: Minus(tP, Times(q, t_prime))}), {t: t_prime}), {s_prime: Minus(sP, Times(q, s_prime))}), {s: s_prime}), {r_prime: Minus(rP, Times(q, r_prime))}), {r: r_prime}), {q: Div(r, r_prime)}))
util = Implies(And(Equals(r_prime, Int(0)), inv), pos)
vc = Implies(pre, And(ini, ForAll([r,r_prime,s,s_prime,t,t_prime,rP,sP,tP], And(pres, util))))
print("<h3>SPC</h3>")
prove(Implies(axioms, vc))